In [1]:
import os
import random
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    recall = recall(y_true, y_pred)
    precision = precision(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [10]:
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.models import load_model

METADATA = '../metadata.npy'
SHAPE = (250, 250, 3)
metadata = np.load(METADATA).item()
base_model = ResNet50(include_top=False, input_shape=SHAPE)
base_model.layers.pop()
model = load_model('../models/CNN_Model.h5', custom_objects={'recall': recall, 'precision': precision, 'f1': f1})

In [12]:
from keras import backend as K
from PIL import Image
from pprint import pprint

labels_dict = {np.argmax(metadata[i]):i for i in metadata.keys()}

img = Image.open("../data/test/1b490196c.png")
img = img.resize(SHAPE[:2][::-1])
img = np.asarray(img, dtype=np.float32)
img = preprocess_input(img.reshape([-1, *SHAPE]))

# Get pred
pp_img = base_model.predict(img)
y = model.predict(pp_img)
label = labels_dict[np.argmax(y)]
info = [(labels_dict[i],j) for i,j in enumerate(y[0])]
pprint(info)

[('Black-grass', 1.3553379e-05),
 ('Charlock', 0.00077064231),
 ('Cleavers', 3.3904216e-05),
 ('Common Chickweed', 0.00024107458),
 ('Common wheat', 5.7182983e-06),
 ('Fat Hen', 3.205123e-05),
 ('Loose Silky-bent', 1.8237231e-05),
 ('Maize', 1.5862446e-06),
 ('Scentless Mayweed', 0.000941479),
 ('Shepherds Purse', 0.99744272),
 ('Small-flowered Cranesbill', 0.00047248436),
 ('Sugar beet', 2.6579539e-05)]
